# Singular value decomposition analysis

In [ ]:
%matplotlib widget
import numpy as np
import matplotlib.pyplot as plt
import DanMAX as DM
style = DM.darkMode(style_dic={'size':'large'})

In [ ]:
# Insert path for the .h5 file - TIP: Use tap for auto-complete
#fname = '/data/visitors/danmax/PROPOSAL/VISIT/raw/SAMPLE/scan-XXXX.h5'
fname = DM.findScan()

# get the azimuthally integrated filename from master file name
aname = DM.getAzintFname(fname)

# read the integrated data
data = DM.getAzintData(aname)
if type(data['q']) != type(None):
    x = data['q']
    Q = True
else:
    x = data['tth']
    Q = False
    
# read common meta data from the master file
meta = DM.getMetaData(fname)

# reduce time-resolution to speed up initial analysis
rf = 1
start = None 
end =  None
data = DM.reduceDic(data,reduction_factor=rf,start=start,end=end)
meta = DM.reduceDic(meta,reduction_factor=rf,start=start,end=end)

I = data['I']
t = meta['time'] # relative time stamp in seconds
T = meta['temp'] # temperature in Kelvin (if available, otherwise None)
I0 = meta['I0']  # relative incident beam intensity "I zero"
E = meta['energy'] # X-ray energy in keV

# normalize the integrated intensity to I0
I = (I.T/I0).T
mask = np.nanmean(I,axis=0)>0
mask[-50:] = False

print(f'Effective time-resolution: {np.mean(np.diff(t)):.2f} s')
U, S, Vh = np.linalg.svd(I[:,mask],True)
components = np.sum((S/np.sum(S))>0.0005)
I_calc = np.matmul(U[:,0:components]*S[0:components],Vh[0:components,:])

In [ ]:

offset = np.arange(0,components,1)
print(f'Component sum: {np.sum(S[:components]/np.sum(S)):.2f}')

fig, [ax0,ax1,ax2] = plt.subplots(1,3)
fig.suptitle(DM.getScan_id(fname))
# Time dependent
ax0.plot(t,(U[:,0:components]+offset))
ax0.set_title('Time dependent')
ax0.set_xlabel('Time (s)')
ax0.set_ylabel('Component')
ax0.invert_yaxis()
# Scale
for i in range(components):
    ax1.plot(i,S[i]/np.sum(S)*100,'.-')
ax1.set_title('Scale')
ax1.set_xlabel('Component')
ax1.set_ylabel('Contribution (%)')

# Radial dependent
ax2.plot(x[mask],Vh[0:components,:].T+offset)
ax2.set_title('Radial dependent')
if Q:
    ax2.set_xlabel('Q (A-1)')
else:
    ax2.set_xlabel('2theta (deg)')
ax2.set_ylabel('Component')
ax2.invert_yaxis()


# observed
fig, [ax0,ax1] = plt.subplots(1,2,sharex=True,sharey=True)
ax0.grid(False), ax1.grid(False)
fig.suptitle(DM.getScan_id(fname))
ax0.pcolormesh(x[mask],t,I[:,mask],norm='log')
ax0.set_title('Observed')
ax0.set_xlabel('2theta (deg)')
ax0.set_ylabel('Time (s)')

# reconstructed
ax1.pcolormesh(x[mask],t,I_calc,norm='log')
ax1.set_title('Reconstructed')
ax1.set_xlabel('2theta (deg)')
ax1.set_ylabel('Time (s)')